# V2 DNA Dataset Creation

## Overview

The following code is used to create a dataset consisting of various different DNA sequences and mutated versions of them.
The mutations used are insertions, deletions and substitutions. They are not combined for the purpose of this dataset. 

It makes use of functions written in the file DNA_Sequence_Generator.py.

The dataset is structured as follows:
- Dataset Directory
    - Excel Spreadsheet with overview of all Collections, and the generation parameters for them. 
    - Collection 1
        - Original Sequence FASTA
        - Sequence with Insertions FASTA
        - Sequence with Deletions FASTA
        - Sequence with Substitutions FASTA
        - Excel Spreadsheet listing the positions and exact nature of each mutation for all the files in the collection.
    - Collection 2
        - Original Sequence FASTA
        - Sequence with Insertions FASTA
        - Sequence with Deletions FASTA
        - Sequence with Substitutions FASTA
        - Excel Spreadsheet listing the positions and exact nature of each mutation for all the files in the collection.
    - Collection n...

The original sequence in each collection differs in terms of its length.

## Changelog

asdasd

## Imports

General Imports

In [10]:
import os
import pandas as pd
from datetime import datetime

Seuqnece Generator Imports

In [11]:
from DNA_Sequence_Generator_V2 import DNA_Sequence_Generator_V2 as DNA
from DNA_Sequence_Generator_V2 import DNA_Sequence_Mutations_V2 as Mutate 

## Dataset Generation

In [12]:
# Define the sequence lengths and the parent directory for the dataset.
sequence_lengths = list(range(1000, 11000, 1000))
parent_directory = "DNA_Sequence_Dataset_V2"

In [13]:
# Creates the parent directory if it doesn't exist.
os.makedirs(parent_directory, exist_ok=True)


In [14]:
# Specify parameters for the DNA sequence generation and subsequent mutation.
gc_content = 0.5
num_mutations = 3
max_bases_per_mutation = 5

In [15]:
# Initializes a list to store all the data for the sequences. 
excel_data = []

In [16]:
# Loop through the sequence lengths, creating a collection for each.
for length in sequence_lengths:
    # Creates a subdirectory for each sequence length.
    subdirectory = os.path.join(parent_directory, f"length_{length}")
    os.makedirs(subdirectory, exist_ok=True)

    # Generates a sequence ID and retrieves the current date.
    sequence_id = f"seq_length_{length}"
    date_of_generation = datetime.today().strftime('%Y-%m-%d')

    # Generates the template (original) sequence.
    original_sequence = DNA.generate_sequence(length, gc_content)

    # Writes the original sequence to a FASTA file.
    original_file_name = f"{sequence_id}_original.fasta"
    DNA.sequence_to_fasta(original_sequence, sequence_id, "original", date_of_generation, file_name=os.path.join(subdirectory, original_file_name))

    # Applies mutations and writes the mutated sequences to FASTA files.
    insertions = Mutate.insert(os.path.join(subdirectory, original_file_name), num_mutations, max_bases_per_mutation)
    deletions = Mutate.delete(os.path.join(subdirectory, original_file_name), num_mutations, max_bases_per_mutation)
    substitutions = Mutate.substitute(os.path.join(subdirectory, original_file_name), num_mutations)

    # Adds the data for this sequence to the master Excel data list.
    excel_data.append([sequence_id, date_of_generation, gc_content, num_mutations, max_bases_per_mutation, insertions, deletions, substitutions])

    # Creates a DataFrame for this collection with a record of all the mutations.
    max_mutations = max(len(insertions), len(deletions), len(substitutions))
    collection_data = {
        original_file_name: [None] * max_mutations,
        f"{sequence_id}_mut_i.fasta": insertions + [None] * (max_mutations - len(insertions)),
        f"{sequence_id}_mut_d.fasta": deletions + [None] * (max_mutations - len(deletions)),
        f"{sequence_id}_mut_s.fasta": substitutions + [None] * (max_mutations - len(substitutions)),
    }
    collection_df = pd.DataFrame(collection_data)
    collection_df.to_excel(os.path.join(subdirectory, f"{sequence_id}_mutations.xlsx"), index=False)

# Check an example output.
collection_df.head()

,seq_length_10000_original.fasta,seq_length_10000_mut_i.fasta,seq_length_10000_mut_d.fasta,seq_length_10000_mut_s.fasta
0,None,3347_3348insA,9346_9349del,1383C>G
1,None,2605_2606insAT,2918_2918del,7677T>C
2,None,1255_1256insC,2687_2690del,1768G>C


In [17]:
# Creates a DataFrame from the master Excel data list for all the different collections.
df = pd.DataFrame(excel_data, columns=["Sequence ID", "Date of Generation", "GC Content", "Number of Mutations", "Max Bases per Mutation", "Insertions", "Deletions", "Substitutions"])
df.head()

,Sequence ID,Date of Generation,GC Content,Number of Mutations,Max Bases per Mutation,Insertions,Deletions,Substitutions
0,seq_length_1000,2024-05-06,0.5,3,5,"[547_548insAACT, 181_182insACCTC, 566_567insGCTA]","[776_776del, 270_272del, 108_110del]","[259C>G, 722T>C, 261C>T]"
1,seq_length_2000,2024-05-06,0.5,3,5,"[85_86insG, 507_508insC, 1635_1636insAA]","[1288_1288del, 1349_1351del, 419_423del]","[56G>C, 1564G>C, 231T>C]"
2,seq_length_3000,2024-05-06,0.5,3,5,"[2266_2267insCTA, 1547_1548insTCAA, 725_726insC]","[1782_1786del, 1546_1550del, 2896_2899del]","[1531A>T, 948A>T, 2657A>C]"
3,seq_length_4000,2024-05-06,0.5,3,5,"[2159_2160insT, 2995_2996insT, 2135_2136insTGC]","[163_167del, 1966_1968del, 2785_2787del]","[3356G>T, 2739A>C, 1511C>A]"
4,seq_length_5000,2024-05-06,0.5,3,5,"[2566_2567insA, 481_482insGCGG, 3715_3716insTG]","[752_752del, 4080_4081del, 822_826del]","[1502G>A, 231G>C, 3738A>G]"


In [18]:
# Saves the DataFrame to an Excel file in the main directory.
excel_file_path = os.path.join(parent_directory, "sequences_info.xlsx")
df.to_excel(excel_file_path, index=False)